### coarse-grain surface elevation correction plot

Show the effects of imposing hydrostatic balance on coarse-grained topography

In [ ]:
from vcm.cubedsphere.coarsen import weighted_block_average
from vcm.convenience import parse_timestep_str_from_path
from vcm import open_tiles
from vcm.catalog import catalog as CATALOG
from matplotlib import pyplot as plt
import matplotlib as mpl
import fv3viz
import os
import fsspec
import numpy as np
import xarray as xr
from dask.diagnostics import ProgressBar

In [ ]:
mpl.rcParams['font.size'] = 8

In [ ]:
RESTARTS_C384_PATH = 'gs://vcm-ml-raw/2020-05-27-40-day-X-SHiELD-simulation-C384-restart-files'
FVCORE='{time:s}/{time:s}.fv_core_coarse.res'
RESTARTS_C48_PATH = '40day_c48_restarts_as_zarr_may2020'
GRID_C48 = CATALOG['grid/c48'].to_dask()
GRID_C384 = CATALOG['grid/c384'].to_dask()
GRAVITY=9.81
SAMPLE_TIMESTEP = '20160805.120000'

In [ ]:
fs, _, _ = fsspec.get_fs_token_paths(RESTARTS_C384_PATH)
timesteps = [parse_timestep_str_from_path(path) for path in fs.ls(RESTARTS_C384_PATH) if path.endswith('00')]
dycore = open_tiles(os.path.join(RESTARTS_C384_PATH, FVCORE.format(time=SAMPLE_TIMESTEP)))

In [ ]:
phis_c48_area_average = weighted_block_average(
    dycore['phis'].rename({'xaxis_1': 'x', 'yaxis_2': 'y'}),
    GRID_C384['area'],
    coarsening_factor=8,
    x_dim='x',
    y_dim='y'
).squeeze()
with ProgressBar():
    phis_c48_area_average = phis_c48_area_average.load()

In [ ]:
restarts = CATALOG['40day_c48_restarts_as_zarr_may2020'].to_dask()
phis_c48_hydrostatic_balance = (
    restarts['phis']
    .sel(time=SAMPLE_TIMESTEP)
    .rename({'grid_yt': 'y', 'grid_xt': 'x'})
    .assign_coords({'tile': list(range(6))})
)
with ProgressBar():
    phis_c48_hydrostatic_balance = phis_c48_hydrostatic_balance.load()

In [ ]:
height_diff = ((phis_c48_hydrostatic_balance - phis_c48_area_average) / GRAVITY).assign_attrs({
    'long_name': 'surface height difference, hydrostatic balance minus area average',
    'units': 'm'
})

In [ ]:
MAPPABLE_VAR_KWARGS = {
    "coord_x_center": "x",
    "coord_y_center": "y",
    "coord_x_outer": "x_interface",
    "coord_y_outer": "y_interface",
    "coord_vars": {
        "lonb": ["y_interface", "x_interface", "tile"],
        "latb": ["y_interface", "x_interface", "tile"],
        "lon": ["y", "x", "tile"],
        "lat": ["y", "x", "tile"],
    },
}

fig, axes, *_ = fv3viz.plot_cube(fv3viz.mappable_var(xr.merge([height_diff, GRID_C48]), 'phis', **MAPPABLE_VAR_KWARGS), vmin=-1500/GRAVITY, cmap='seismic')
axes.item().set_title(SAMPLE_TIMESTEP)
fig.set_size_inches([3.8, 2.5])
fig.savefig('figures/Figure_A1_hydrostatic_terrain_correction.eps', bbox_inches='tight')

In [ ]:
# this converts matplotlib eps files to a more manageable size

!epstopdf figures/Figure_A1_hydrostatic_terrain_correction.eps
!pdftops -eps figures/Figure_A1_hydrostatic_terrain_correction.pdf
!rm figures/Figure_A1_hydrostatic_terrain_correction.pdf